![TrainingNotebookLogo.png](https://downloads.limelightvision.io/content/TrainingNotebookLogo.png)

To train a neural object detector for Limelight, click the "play" button on each code block. Pay extra attention to any "❗" you see. By the end of this tutorial, you will have downloaded a .zip file containing your model and label files.

See https://docs.limelightvision.io/docs/docs-limelight/pipeline-neural/training-your-own-detector for a more in-depth tutorial.

# 1. Install The Object Detection Package


In [1]:
import shutil
import os
tmpModelPath ='/content/models'
if os.path.exists(tmpModelPath) and os.path.isdir(tmpModelPath):
  shutil.rmtree(tmpModelPath)

MLENVIRONMENT="COLAB"
!git clone --depth 1 https://github.com/tensorflow/models
!cd models && git fetch --depth 1 origin ad1f7b56943998864db8f5db0706950e93bb7d81 && git checkout ad1f7b56943998864db8f5db0706950e93bb7d81

fatal: destination path 'models' already exists and is not an empty directory.
remote: Enumerating objects: 3055, done.
remote: Counting objects: 100% (3055/3055), done.
remote: Compressing objects: 100% (1359/1359), done.
remote: Total 1824 (delta 1223), reused 703 (delta 446), pack-reused 0 (from 0)
Receiving objects: 100% (1824/1824), 10.05 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (1223/1223), completed with 739 local objects.
From https://github.com/tensorflow/models
 * branch            ad1f7b56943998864db8f5db0706950e93bb7d81 -> FETCH_HEAD
HEAD is now at ad1f7b5 adjust folder path


In [ ]:
# Environment Setup
import os
import sys
import re

MLENVIRONMENT="COLAB"
HOMEFOLDER = '/root/MultiAgent/FTCTraining/'
os.environ["HOMEFOLDER"] = HOMEFOLDER
FINALOUTPUTFOLDER_DIRNAME = 'final_output'
FINALOUTPUTFOLDER = os.path.join(HOMEFOLDER, FINALOUTPUTFOLDER_DIRNAME)

# Create required directories
os.makedirs(os.path.join(HOMEFOLDER, 'models/research'), exist_ok=True)
os.makedirs(os.path.join(HOMEFOLDER, 'models/mymodel'), exist_ok=True)

# Update tf_slim path
tf_slim_path = '/root/miniconda/envs/FTCTrainingEnv/lib/python3.11/site-packages/tf_slim/data/tfexample_decoder.py'

# Check if file exists before modifying
if os.path.exists(tf_slim_path):
    with open(tf_slim_path, 'r') as file:
        content = file.read()
        content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
        content = re.sub(r'control_flow_ops.case', 'tf.case', content)
        content = re.sub(r'control_flow_ops.cond', 'tf.compat.v1.cond', content)
    with open(tf_slim_path, 'w') as file:
        file.write(content)
    print(f"File {tf_slim_path} updated successfully")
else:
    print(f"Warning: Could not find {tf_slim_path}")

print(f"HOMEFOLDER set to: {HOMEFOLDER}")
print(f"Current working directory: {os.getcwd()}")

print(sys.version)
if(MLENVIRONMENT == "COLAB"):
    print("colab env setup")
    os.environ["HOMEFOLDER"] = "/root/MultiAgent/FTCTraining/"
    HOMEFOLDER = '{HOMEFOLDER}'.format(**os.environ)
    FINALOUTPUTFOLDER_DIRNAME = 'final_output'
    FINALOUTPUTFOLDER = HOMEFOLDER+FINALOUTPUTFOLDER_DIRNAME
    print(HOMEFOLDER)

# Copy setup files into models/research folder
!cd {HOMEFOLDER}models/research && pwd && protoc object_detection/protos/*.proto --python_out=.

# Modify setup.py
with open(HOMEFOLDER+'models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open(HOMEFOLDER+'models/research/setup.py', 'w') as f:
    if(MLENVIRONMENT == "COLAB"):
        s = re.sub('tf-models-official>=2.5.1','tf-models-official==2.15.0', s)
        f.write(s)

3.11.11 (main, Dec 11 2024, 16:28:39) [GCC 11.2.0]
colab env setup
/content/
/bin/bash: line 1: cd: /content/models/research: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: '/content/models/research/object_detection/packages/tf2/setup.py'

In [ ]:
# Install
!pip install {HOMEFOLDER}models/research/

Test the environment by running `model_builder_tf2_test.py` to make sure everything is working as expected.

In [1]:
!python {HOMEFOLDER}models/research/object_detection/builders/model_builder_tf2_test.py

python: can't open file '/root/MultiAgent/FTCTraining/{HOMEFOLDER}models/research/object_detection/builders/model_builder_tf2_test.py': [Errno 2] No such file or directory


#2. Get Dataset From Google Drive

❗Using the same Google account you are using for this Colab, upload your RoboFlow .tfrecord.zip to your Google Drive. Some input is required in this section❗

This code block will mount your entire Google Drive in the "Files" pane on the left-hand side of your screen.

❗Click the Refresh button in the "Files" pane after running the code block.❗

In [ ]:
if(MLENVIRONMENT=="COLAB"):
    from google.colab import drive
    drive.mount('/content/gdrive')

❗ Run the code block, wait for the file selection interface to appear, and select your .tfrecord.zip dataset ❗


In [ ]:
if(MLENVIRONMENT=="COLAB"):
    datasetPath = '\''+fc.selected+ '\''
    print(datasetPath)
    !unzip $datasetPath

Auto-detect relevant tfrecord components

In [ ]:
import os
import fnmatch

def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename

def set_tfrecord_variables(directory):
    train_record_fname = ''
    val_record_fname = ''
    label_map_pbtxt_fname = ''

    for tfrecord_file in find_files(directory, '*.tfrecord'):
        if '/train/' in tfrecord_file:
            train_record_fname = tfrecord_file
        elif '/valid/' in tfrecord_file:
            val_record_fname = tfrecord_file
        elif '/test/' in tfrecord_file:
            pass

    for label_map_file in find_files(directory, '*_label_map.pbtxt'):
        label_map_pbtxt_fname = label_map_file  # Assuming one common label map file

    return train_record_fname, val_record_fname, label_map_pbtxt_fname

train_record_fname, val_record_fname, label_map_pbtxt_fname = set_tfrecord_variables('/content')

#if(MLENVIRONMENT=="COLAB"):
    #train_record_fname = '/content/train/cubes-cones.tfrecord'
    #val_record_fname = '/content/valid/cubes-cones.tfrecord'
    #label_map_pbtxt_fname = '/content/train/cubes-cones_label_map.pbtxt'

print("Train Record File:", train_record_fname)
print("Validation Record File:", val_record_fname)
print("Label Map File:", label_map_pbtxt_fname)



# 3.&nbsp;Training Configuration and Labels File Generation

Download the pre-trained Limelight Base Model

In [ ]:
chosen_model = 'ssd-mobilenet-v2'
MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
}
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

# Create "mymodel" folder for pre-trained weights and configuration files
%cd ~
%mkdir {HOMEFOLDER}models/mymodel/
%cd {HOMEFOLDER}models/mymodel/
%pwd

# Download pre-trained model weights
import tarfile
download_tar = 'https://downloads.limelightvision.io/models/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://downloads.limelightvision.io/models/' + base_pipeline_file
!wget {download_config}
%cd ~

# Set training parameters for the model
num_steps = 40000
checkpoint_every = 2000
batch_size = 16


Generate Labels File

In [ ]:

# Set file locations and get number of classes for config file
pipeline_fname = HOMEFOLDER+'models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = HOMEFOLDER+'models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

def get_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    class_names = [category['name'] for category in category_index.values()]
    return class_names

def create_label_file(filename, labels):
    with open(filename, 'w') as file:
        for label in labels:
            file.write(label + '\n')


num_classes = get_num_classes(label_map_pbtxt_fname)
classes = get_classes(label_map_pbtxt_fname)

print('Total classes:', num_classes)
print(classes)


#Generate labels file
create_label_file(HOMEFOLDER + "limelight_neural_detector_labels.txt", classes)

Modify the base Limelight Model Configuration File

Augmentation Options: https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto

In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

print('writing custom configuration file')



with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('checkpoint_every_n: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .004', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .0016666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)

# (Optional) Display the custom configuration file's contents
# !cat pipeline_file.config
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = 'pipeline_file.config'
model_dir = HOMEFOLDER+'training_progress/'
print(" ")
print(model_dir)

# 4.&nbsp;Train Model

Once training starts, come back and click the refresh button within the tensorboard window to check training progress.



In [26]:
%load_ext tensorboard
%tensorboard --logdir '/content/training_progress/train'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6590), started 0:13:28 ago. (Use '!kill 6590' to kill it.)

Fix TF 2.15 breaking changes

In [ ]:
import shutil
import re

original_path = '/usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py'
with open(original_path, 'r') as file:
  content = file.read()
  content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
  content = re.sub(r'control_flow_ops.case', 'tf.case', content)
  content = re.sub(r'control_flow_ops.cond', 'tf.compat.v1.cond', content)
with open(original_path, 'w') as file:
  file.write(content)

print(f"File {original_path} fixed.")

Train

In [ ]:
!rm -rf {HOMEFOLDER}training_progress
# Run training!
!python {HOMEFOLDER}models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --checkpoint_every_n={checkpoint_every} \
    --num_train_steps={num_steps} \
    --num_workers=2 \
    --sample_1_of_n_eval_examples=1

In [31]:
# Fix 1: Correct TF 2.15 compatibility patch path
tf_slim_path = '/root/miniconda/envs/FTCTrainingEnv/lib/python3.11/site-packages/tf_slim/data/tfexample_decoder.py'
# Update these 3 paths in your notebook
train_record_fname = '/root/MultiAgent/FTCTraining/train/samples-specimens.tfrecord'
val_record_fname = '/root/MultiAgent/FTCTraining/valid/samples-specimens.tfrecord'
label_map_pbtxt_fname = '/root/MultiAgent/FTCTraining/test/samples-specimens_label_map.pbtxt'  # Rename .bbtxt to .pbtxt
if os.path.exists(tf_slim_path):
    with open(tf_slim_path, 'r') as file:
        content = file.read()
        content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
        content = re.sub(r'control_flow_ops\.case', 'tf.case', content)
        content = re.sub(r'control_flow_ops\.cond', 'tf.compat.v1.cond', content)
    with open(tf_slim_path, 'w') as file:
        file.write(content)

# Fix 2: Update pipeline.config modifications
def update_pipeline_config():
    with open(pipeline_fname) as f:
        s = f.read()
    
    replacements = {
        'fine_tune_checkpoint: ".*?"': f'fine_tune_checkpoint: "{fine_tune_checkpoint}"',
        'input_path: ".*?PATH_TO_BE_CONFIGURED/train.*?"': f'input_path: "{train_record_fname}"',
        'input_path: ".*?PATH_TO_BE_CONFIGURED/val.*?"': f'input_path: "{val_record_fname}"',
        'label_map_path: ".*?"': f'label_map_path: "{label_map_pbtxt_fname}"',
        'batch_size: [0-9]+': f'batch_size: {batch_size}',
        'num_steps: [0-9]+': f'num_steps: {num_steps}',
        'num_classes: [0-9]+': f'num_classes: {num_classes}'
    }

    for pattern, replacement in replacements.items():
        s = re.sub(pattern, replacement, s)

    with open('pipeline_file.config', 'w') as f:
        f.write(s)

update_pipeline_config()

# Fix 3: Add path validation before training
def validate_paths():
    required_files = {
        "Training TFRecord": train_record_fname,
        "Validation TFRecord": val_record_fname,
        "Label Map": label_map_pbtxt_fname,
        "Pipeline Config": pipeline_file
    }
    
    missing = False
    for name, path in required_files.items():
        if not os.path.exists(path):
            print(f"❌ Missing {name}: {path}")
            missing = True
        else:
            print(f"✅ Found {name}: {path}")
    
    if missing:
        raise FileNotFoundError("Missing required training files. Check paths above.")

validate_paths()

# Fix 4: Update training command with proper paths
!python {os.path.join(HOMEFOLDER, 'models/research/object_detection/model_main_tf2.py')} \
    --pipeline_config_path={os.path.abspath(pipeline_file)} \
    --model_dir={os.path.abspath(model_dir)} \
    --alsologtostderr \
    --checkpoint_every_n={checkpoint_every} \
    --num_train_steps={num_steps} \
    --num_workers=2 \
    --sample_1_of_n_eval_examples=1

✅ Found Training TFRecord: /root/MultiAgent/FTCTraining/train/samples-specimens.tfrecord
✅ Found Validation TFRecord: /root/MultiAgent/FTCTraining/valid/samples-specimens.tfrecord
✅ Found Label Map: /root/MultiAgent/FTCTraining/test/samples-specimens_label_map.pbtxt
✅ Found Pipeline Config: pipeline_file.config
2025-01-31 11:22:29.523925: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 11:22:29.551508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-31 11:22:29.551568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one 

Feel free to stop training early. Check the 'training_progress' folder to see all training checkpoints.


# 5.&nbsp;Convert Model to TFLite

In [ ]:
#remove final output folder if it exists
if os.path.exists(FINALOUTPUTFOLDER) and os.path.isdir(FINALOUTPUTFOLDER):
  shutil.rmtree(FINALOUTPUTFOLDER)

# Make a directory to store the trained TFLite model
!mkdir {FINALOUTPUTFOLDER}
print(FINALOUTPUTFOLDER)
# Export graph
# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = HOMEFOLDER+'training_progress'
exporter_path = HOMEFOLDER+'models/research/object_detection/export_tflite_graph_tf2.py'
output_directory = FINALOUTPUTFOLDER

!python $exporter_path \
    --trained_checkpoint_dir $last_model_path \
    --output_directory $output_directory \
    --pipeline_config_path $pipeline_file

# Convert to .tflite Flatbuffer
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
tflite_model = converter.convert()
model_path_32bit = FINALOUTPUTFOLDER+'/limelight_neural_detector_32bit.tflite'
with open(model_path_32bit, 'wb') as f:
  f.write(tflite_model)

!cp {HOMEFOLDER}limelight_neural_detector_labels.txt {FINALOUTPUTFOLDER}
!cp {HOMEFOLDER}models/mymodel/pipeline_file.config {FINALOUTPUTFOLDER}

In [ ]:
# Export graph
# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = HOMEFOLDER+'training_progress'
exporter_path = HOMEFOLDER+'models/research/object_detection/export_tflite_graph_tf2.py'
output_directory = FINALOUTPUTFOLDER

!python $exporter_path \
    --trained_checkpoint_dir $last_model_path \
    --output_directory $output_directory \
    --pipeline_config_path $pipeline_file

# Convert to .tflite Flatbuffer
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
tflite_model = converter.convert()
model_path_32bit = FINALOUTPUTFOLDER+'/limelight_neural_detector_32bit.tflite'
with open(model_path_32bit, 'wb') as f:
  f.write(tflite_model)

!cp {HOMEFOLDER}limelight_neural_detector_labels.txt {FINALOUTPUTFOLDER}

# 6. Quantize model
The "TFLiteConverter" module will perform [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) on the model. To quantize the model, we need to provide a set of example images. We will extract 100 images from the training tfrecord and place said images into the "extracted_samples" folder.


In [ ]:
import tensorflow as tf
import os
import io
from PIL import Image

def extract_images_from_tfrecord(tfrecord_path, output_folder, num_samples=100):
    # Make sure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize a counter for the number of images saved
    saved_images = 0

    # Read the TFRecord file
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
    for raw_record in raw_dataset.take(num_samples):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())

        # Extract the image data (change 'image/encoded' if necessary)
        image_data = example.features.feature['image/encoded'].bytes_list.value[0]

        # Decode the image data and save as a file
        image = Image.open(io.BytesIO(image_data))
        image.save(os.path.join(output_folder, f'image_{saved_images}.png'))

        saved_images += 1
        if saved_images >= num_samples:
            break

    print(f"Extracted {saved_images} images to {output_folder}")

# Set the path to your TFRecord file and the output directory
tfrecord_path = train_record_fname
extracted_sample_folder = HOMEFOLDER+'extracted_samples'

#remove sample folder if it exists
if os.path.exists(extracted_sample_folder) and os.path.isdir(extracted_sample_folder):
  shutil.rmtree(extracted_sample_folder)

# Extract images
extract_images_from_tfrecord(tfrecord_path, extracted_sample_folder)


# Get list of all images in train directory
from google.cloud import storage
import glob

quant_image_list=[]
if(MLENVIRONMENT=="COLAB"):

    jpg_file_list = glob.glob(extracted_sample_folder + '/*.jpg')
    jpeg_file_list = glob.glob(extracted_sample_folder + '/*.jpeg')
    JPG_file_list = glob.glob(extracted_sample_folder + '/*.JPG')
    png_file_list = glob.glob(extracted_sample_folder + '/*.png')
    bmp_file_list = glob.glob(extracted_sample_folder + '/*.bmp')
    quant_image_list = jpg_file_list + JPG_file_list + png_file_list + bmp_file_list

print("pulling samples from " + extracted_sample_folder)
print("samples: " + str(len(quant_image_list)))

In [ ]:
# A generator that provides a representative dataset
# Code modified from https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_classification_ptq_tf2.ipynb

# First, get input details for model so we know how to preprocess images
interpreter = tf.lite.Interpreter(model_path=model_path_32bit)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

import random

def representative_data_gen():
  dataset_list = quant_image_list
  quant_num = 300
  for i in range(quant_num):
    pick_me = random.choice(dataset_list)
    print(pick_me)
    image = tf.io.read_file(pick_me)

    if pick_me.endswith('.jpg') or pick_me.endswith('.JPG') or pick_me.endswith('.jpeg'):
      image = tf.io.decode_jpeg(image, channels=3)
    elif pick_me.endswith('.png'):
      image = tf.io.decode_png(image, channels=3)
    elif pick_me.endswith('.bmp'):
      image = tf.io.decode_bmp(image, channels=3)

    image = tf.image.resize(image, [width, height])  # TO DO: Replace 300s with an automatic way of reading network input size
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

Finally, we'll initialize the TFLiteConverter module, point it at the TFLite graph we generated in Step 6, and provide it with the representative dataset generator function we created in the previous code block. We'll configure the converter to quantize the model's weight values to INT8 format.

In [ ]:
# Initialize converter module
converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
print("initialized converter")
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input tensors to uint8 and output tensors to float32
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32
print("begin conversion")
tflite_model = converter.convert()
print("conversion complete")

with open(FINALOUTPUTFOLDER+'/limelight_neural_detector_8bit.tflite', 'wb') as f:
  f.write(tflite_model)

# 7. Compile Model for Limelight & Download


Install Coral Compiler

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
! sudo apt-get update
! sudo apt-get install edgetpu-compiler

Compile the previously-generated 8-bit model for Google Coral

In [ ]:
!cd {FINALOUTPUTFOLDER} && pwd && edgetpu_compiler limelight_neural_detector_8bit.tflite && pwd && mv limelight_neural_detector_8bit_edgetpu.tflite limelight_neural_detector_coral.tflite && rm limelight_neural_detector_8bit_edgetpu.log

Zip models

In [ ]:
!rm {HOMEFOLDER}limelight_detectors.zip
!zip -r {HOMEFOLDER}limelight_detectors.zip {FINALOUTPUTFOLDER}

Download

In [ ]:
from google.colab import files
files.download(HOMEFOLDER+'limelight_detectors.zip')